# OLD notebook
# Task 2

In [1]:
import numpy as np
import strid
import matplotlib.pyplot as plt
import scipy.signal
import functions as fun

/Users/emilneby/miniforge3/envs/tkt4550/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Perform covSSI on generated data to get the estimated modes
stochastic_data_path = "results/data-stochastic_8_floor.npz"
combined_data_path = "results/data-combined_8_floor.npz"
ssid, modes = fun.covSSI(stochastic_data_path)

In [3]:
%matplotlib nbagg
stabdiag = strid.StabilizationDiagram()
modes = stabdiag.plot(modes)

f, psd = ssid.psdy(nperseg=2**10)
stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)
plt.savefig("figures/cov_SSI_output.jpg", dpi=300)

<IPython.core.display.Javascript object>

### Task 2.1 - finding relative difference

In [4]:
cluster_meat = fun.rel_difference(modes)
# !!! Have to handle modes in the lowest order!!!

In [5]:
print(np.max(cluster_meat))

0.9999898744335807


### Task 2.2 -  Using K-means clustering to separate all the poles into two groups

In [6]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(cluster_meat)
labels = kmeans.labels_

In [7]:
#Assign label to each mode
physical_coordinates = []
mathematical_coordinates = []
count = 0
physical_modes_dict = {}
physical_modes_list = []
num_modes = 0
for order in range(49, 5, -1):
    modes_of_order = modes[order]
    #print("Order: " + str(order))
    physical_modes_in_order = []
    for mode in modes_of_order:
        mode.physical = labels[count]
        if (mode.physical == 0):
            physical_modes_in_order.append(mode)
            physical_modes_list.append(mode)
            physical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        else:
            mathematical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        count += 1
    physical_modes_dict[order] = physical_modes_in_order
    num_modes += len(physical_modes_in_order)

physical_coordinates = np.array(physical_coordinates)
mathematical_coordinates = np.array(mathematical_coordinates)
physical_modes_list = np.array(physical_modes_list)


In [8]:
print(physical_modes_list.shape[0])

555


In [9]:
#Making a 3D scatterplot of frequency, damping and MAC number divided into physical and mathematical clusters
fig = plt.figure()
ax = plt.axes(projection='3d')

fg = ax.scatter3D(physical_coordinates[:,0], physical_coordinates[:,1], physical_coordinates[:,2], c="blue", label= "Physical modes")
ax.scatter3D(mathematical_coordinates[:,0], mathematical_coordinates[:,1], mathematical_coordinates[:,2], c="red", label="Mathematical modes")
ax.legend()
ax.set_xlabel("Frequency")
ax.set_ylabel("Damping")
ax.set_zlabel("MAC")
plt.savefig("figures/3dscatter.jpg", dpi=300)

print("Number of physical modes = " + str(physical_coordinates.shape[0]))
print("Number of mathematical modes = " +str(mathematical_coordinates.shape[0]))

<IPython.core.display.Javascript object>

Number of physical modes = 555
Number of mathematical modes = 52


In [10]:
%matplotlib nbagg
stabdiag = strid.StabilizationDiagram()
stabdiag.plot_partition_cluster(modes)

f, psd = ssid.psdy(nperseg=2**10)
stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)
plt.savefig("figures/first_kmeans_output.jpg", dpi=300)

<IPython.core.display.Javascript object>

Length of color list: 302


# Task 3
### Task 3.1 Detecting structural modes by hierachical clustering
Using hierarchical clustering with a signle linkage method with a cut-off-distance of $d_c=0.04$
$$ d_{c_{i,j}} = d \lambda_{i,j} + (1- MAC_{i,j}) $$

In [11]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
# Plotting dendogram: https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html
# https://www.youtube.com/watch?v=v7oLMvcxgFY&ab_channel=KindsonTheTechPro

import strid
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering


#Since we want to define our own distance between two poles, we must compute a distance matrix we can feed into the clustering algorithm.


In [12]:
distance = fun.distance_matrix(physical_modes_list)

/Users/emilneby/PyCharmProjects/TKT4550_Prosjekt/exercises/functions.py:177: ComplexWarning: Casting complex values to real discards the imaginary part
  dist_matrix[i, j] = (np.abs((eigen_i - eigen_j)) / np.max([eigen_i, eigen_j])) + (


Distance matrix computational time = 4.503363132476807sec


In [13]:
#Check the integrity of the distance matrix
print(np.max(distance))
print(np.mean(distance))
print(np.min(distance))

1.544131433929679
0.8776549673649396
0.0


In [14]:
#Perform the actual clustering
model = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='average', distance_threshold=0.04)
#The agglomerative clustering algorithm only finds 1 non-structural mode with single linkage method. Works better so far with 'complete' or 'average'.

In [15]:
y_hc = model.fit_predict(distance)
model = model.fit(distance)
#print((y_hc))
print(np.max(y_hc))

117


In [16]:
hierarchy = y_hc.tolist()
print(len(y_hc))
print(type(hierarchy))
#print(hierarchy)

num_modes_in_hierarchy = np.zeros((np.max(y_hc) + 1, 3))
for i in range(0, len(num_modes_in_hierarchy)):
    num_modes_in_hierarchy[i, 0] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 1] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 2] = i

print((num_modes_in_hierarchy))

555
<class 'list'>
[[  3.   3.   0.]
 [  2.   2.   1.]
 [ 18.  18.   2.]
 [  3.   3.   3.]
 [  3.   3.   4.]
 [  5.   5.   5.]
 [  2.   2.   6.]
 [  3.   3.   7.]
 [  4.   4.   8.]
 [  2.   2.   9.]
 [  3.   3.  10.]
 [ 37.  37.  11.]
 [  2.   2.  12.]
 [  3.   3.  13.]
 [  3.   3.  14.]
 [  2.   2.  15.]
 [  2.   2.  16.]
 [  2.   2.  17.]
 [  3.   3.  18.]
 [  2.   2.  19.]
 [  2.   2.  20.]
 [ 36.  36.  21.]
 [  3.   3.  22.]
 [  6.   6.  23.]
 [  2.   2.  24.]
 [ 10.  10.  25.]
 [  2.   2.  26.]
 [  2.   2.  27.]
 [  2.   2.  28.]
 [ 43.  43.  29.]
 [  3.   3.  30.]
 [  2.   2.  31.]
 [ 43.  43.  32.]
 [  1.   1.  33.]
 [  6.   6.  34.]
 [  1.   1.  35.]
 [  1.   1.  36.]
 [  3.   3.  37.]
 [  2.   2.  38.]
 [  3.   3.  39.]
 [  3.   3.  40.]
 [  1.   1.  41.]
 [  2.   2.  42.]
 [  2.   2.  43.]
 [  5.   5.  44.]
 [  3.   3.  45.]
 [  3.   3.  46.]
 [  3.   3.  47.]
 [ 10.  10.  48.]
 [  2.   2.  49.]
 [ 41.  41.  50.]
 [  3.   3.  51.]
 [  1.   1.  52.]
 [  2.   2.  53.]
 [  6.   

In [17]:
structural_coordinates = []
non_structural_coordinates = []

for i in range(0, len(y_hc)):
    if y_hc[i] == 1:
        structural_coordinates.append(physical_coordinates[i,:])
    else:
        non_structural_coordinates.append(physical_coordinates[i,:])

structural_coordinates = np.array(structural_coordinates)
non_structural_coordinates = np.array(non_structural_coordinates)

In [18]:
#Assign hierarchy to each mode
count = 0
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    for mode in modes_of_order:
        mode.cluster = hierarchy[count]
        count += 1

### Task 3.2 Need to visualize the hierarchical clusters

# Task 4
### Task 4.1 - Using k-means to divide into one cluster with "many modes" (structural modes) and one cluster with "few/scattered modes" (mathematcal modes).

In [19]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(num_modes_in_hierarchy[:,:2])
labels2 = kmeans.labels_

In [20]:
#print(labels2)
print("Number of clusters identified: " +str(len(labels2)))
#print((labels2))

Number of clusters identified: 118


In [21]:
# Pick the label with lowest occurence

labels2 = labels2.tolist()

structura_label = None
if (labels2.count(0) > labels2.count(1)):
    structural_label = 1
else:
    structural_label = 0



In [22]:
structural_hierarchies = []

for i in range(0, len(labels2)):
    if(labels2[i] == structural_label):
        structural_hierarchies.append(i)

print(structural_hierarchies)
#need to create a dictionary with colors to the clusters

[11, 21, 29, 32, 50, 79, 96, 103]


In [23]:
#Assign hierarchy to each mode
structural_modes_dict = {}
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    structural_modes_in_order = []
    for mode in modes_of_order:
        #print(mode.f)
        if mode.cluster in structural_hierarchies:
            structural_modes_in_order.append(mode)

    structural_modes_dict[order] = structural_modes_in_order

In [24]:
%matplotlib nbagg
#Making a new stabilization diagram with the physical modes
stabdiag = strid.StabilizationDiagram()
stabdiag.plot_hierarchies(structural_modes_dict)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>

Length of color list: 302


### Task 4.2 - Extract modal features of each detected mode as the average of all he components' features within each hierarchical cluster.

In [259]:

clustered_modes = fun.HierarchicalModes(stabdiag)

modes_in_clusters = clustered_modes.clusters_dict(structural_modes_dict)

In [260]:
#Verfying numbers of clusters up against stabilization diagram
#print(len(modes_in_clusters))
#clusters = (list(modes_in_clusters.values()))

In [261]:
from collections import defaultdict

hierarchies = sorted([*modes_in_clusters.keys()])
clusters = defaultdict(list)

for hierarchy in hierarchies:
    frequencies_in_hierarchy = []
    damping_in_hierarchy = []
    mode_shapes_in_hierarchy = []
    for element in modes_in_clusters[hierarchy]:
        frequencies_in_hierarchy.append(element.f)
        damping_in_hierarchy.append(element.xi)
        mode_shapes_in_hierarchy.append(element.v)

    f_mean = np.mean(np.array(frequencies_in_hierarchy))
    xi_mean = np.mean(np.array(damping_in_hierarchy))
    modal_shapes_mean = np.mean(np.array(mode_shapes_in_hierarchy), axis=0)

    clusters[hierarchy] = [f_mean, xi_mean, modal_shapes_mean]

clustered_features = dict(clusters)

In [262]:
#Sort the clusters by frequency to group into modes
sorted_features_dict = {cluster: mode for cluster, mode in sorted(clustered_features.items(), key=lambda item: item[1][0])}

sorted_features_list = np.array(list(sorted_features_dict.values()))

/var/folders/xg/ddkjb2x96w167hqnv_gjv_v40000gn/T/ipykernel_22098/3669789805.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sorted_features_list = np.array(list(sorted_features_dict.values()))


In [263]:
est_frequencies = sorted_features_list[:,0]
est_damping = sorted_features_list[:,1]
est_modeshapes = sorted_features_list[:,2]

In [264]:
print(est_frequencies)
print(est_damping)
print(est_modeshapes)

[2.9839965112516893 8.703529885652598 14.198555372520618 19.1886475771783
 23.440418525012582 27.116949571739962 29.711092591451155
 31.33904943238427]
[0.04742007563464635 0.027135810238762124 0.029901192053885643
 0.03303734080753947 0.04358653146231284 0.04088293015408231
 0.050803349828730755 0.04975057990758646]
[array([ 0.00105958+0.01767807j, -0.00358119+0.03254295j,
        -0.00359993+0.04350477j, -0.01275056+0.06083252j,
        -0.00894786+0.07375762j, -0.01442224+0.07454926j,
        -0.0091015 +0.08734727j, -0.01711316+0.09332897j])
 array([-0.00551506+0.02349448j, -0.010613  +0.03934029j,
        -0.01116764+0.04511891j, -0.00989288+0.0360251j ,
        -0.00369251+0.01719771j,  0.00223587-0.00870691j,
         0.00750991-0.02985937j,  0.0104525 -0.04273473j])
 array([-0.00305994+0.00860382j, -0.00430445+0.01048609j,
        -0.00159218+0.00352184j,  0.00208544-0.00532912j,
         0.00385395-0.01024767j,  0.00314634-0.00693832j,
        -0.00104286+0.00167366j, -0.00371

### Task 4.3 - Verify against ground truth

In [265]:
data = np.load("results/data-stochastic_8_floor.npz")

true_f = data["true_frequencies"]
true_xi = data["true_damping"]
true_modeshapes = data["true_modeshapes"]

In [266]:
print(true_f)

[ 2.9369931   8.71096352 14.18829253 19.18245625 23.52338443 27.06325197
 29.68151292 31.28900555]


In [267]:
print(np.mean(np.abs(true_f-est_frequencies)))

0.03589728501405698


In [268]:
fig = plt.figure("Damping estimate")
axd = fig.add_axes((0.1, 0.1, .8, .8))
axd.set(xlabel='Frequency', ylabel='Damping ratio', title='Estimated and true frequency and damping',
        ylim=(0, .10)
       )
figmodes, axes = plt.subplots(ncols=4, nrows=2, dpi=144)
res = []
for n in range(true_f.size):
    ax = axes.flatten()[n]
    un = true_modeshapes[n]
    fn = true_f[n]
    xin = true_xi[n]
    #nmax = np.argmax([strid.modal_assurance_criterion(mode.v, un) for mode in modes])
    #mode = modes[nmax]

    line, = axd.plot(est_frequencies[n], est_damping[n], 'x')
    line, = axd.plot(fn, xin, 'o', mec=line.get_color(), mfc=(0, 0, 0, 0))


    ferr = (est_frequencies[n]-fn)/fn * 100
    xierr = (est_damping[n]-xin)/xin*100
    mac = strid.modal_assurance_criterion(un, est_modeshapes[n])
    res.append([n, ferr, xierr, mac*100,])


    v_true = np.r_[0., un]
    v = np.r_[0, est_modeshapes[n]]
    v = strid.modal_scale_factor(v, v_true)*v
    z = np.arange(v.size)
    ax.plot(v_true, z, label='True')
    ax.plot(v.real, z, label='Estimated')

    if n == 2:
        ax.legend(bbox_to_anchor=(.5, 1.20), loc='lower center', ncol=2)
        axd.legend(['Estimated', 'True'],)
    ax.axvline(0., color=(0, 0, 0, .3))

    ax.set_title(f"Mode {n}")
    ax.axis('off')
    ax.set_xlim(-.5, .5)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>